In [9]:
import numpy as np
import os
from os.path import join
from glob import iglob
from shutil import copyfile

In [2]:
root_path = '/data/ksb'
matchsum_root = join(root_path, 'MatchSum/MatchSum')
gsum_root = join(root_path, 'GSum/guided_summarization')
gsum_bart_root = join(gsum_root, 'bart')
bert_root = join(root_path, 'BertSum/PreSumm')

In [23]:
data_root = join(root_path, 'cnn-dailymail/finished_files')
url_mapping = join(root_path, 'cnn-dailymail/url_lists')

In [24]:
bert_data = join(bert_root, 'raw_stories')
bert_url_mapping = join(bert_root, 'urls')

In [ ]:
match_data_dir = join(matchsum_root, 'data')
match_roberta_dir = join(matchsum_root,'roberta/result')

In [10]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as err:
        if err.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
def del_folder(path):
    try:
        rmtree(path)
    except:
        pass

### BERT Extractor

In [17]:
del_folder(bert_data)
mkdir_p(bert_data)

In [18]:
len(list(iglob(join(data_root, '**/**.story'), recursive=False)))

312085

아래 코드 사용하지 않음  
BertSum/PreSumm/src/prepo/data_builder.py 의 raw_path 수정

In [25]:
for urlfile_path in iglob(join(url_mapping, 'all_**.txt'), recursive=False):
    try:
        up = urlfile_path.split(os.sep)[-1]
        urlfile_name = up.replace('all','mapping')
        
        copyfile(urlfile_path, join(bert_url_mapping, urlfile_name))
        
    except Exception as e:
        print(e)

In [56]:
def get_bert_urlfilename(path):
    bert_filename = ''
    urlfilename = path.split(os.sep)[-1].replace('all_','')
    urlfilename = urlfilename[:-4]
    
    if urlfilename == 'train':
        bert_filename = 'mapping_train.txt'
    elif urlfilename == 'test':
        bert_filename = 'mapping_test.txt'
    elif urlfilename == 'val':
        bert_filename = 'mapping_valid.txt'
    else:
        raise ValueError
        
    return bert_filename

In [59]:
from difflib import context_diff


bert_origin_urlpath = join(bert_url_mapping,'origin_')
    
for urlfile_path in iglob(join(url_mapping, 'all_**.txt'), recursive=False):
    
    bert_urlfile_name = get_bert_urlfilename(urlfile_path)
        
    with open(urlfile_path, 'r') as f_all:
        with open(join(bert_origin_urlpath, bert_urlfile_name), 'r') as f_bert:
            diff = context_diff(f_all.readlines(), f_bert.readlines(), fromfile='f_all', tofile='f_bert')
            assert not list(diff) 
            

### MatchSum Extractor

In [ ]:
def copy_file(src_regex, dst_path):
    for src in iglob(src_regex, recursive=False):
        copyfile(src, dst_path)

In [ ]:
bert_result = join(matchsum_root, 'results')
split_symbol = '<q>'

In [ ]:
def get_result_list(src_regex):
    gold, cand, cand_num, src = [], [], [], []

    for file_paths in iglob(src_regex, recursive=False):
        lines = []
        with open(file_paths) as f:
            lines = f.readlines()

        if file_paths.endswith('gold'):
            gold = lines
        elif file_paths.endswith('candidate_sent_num'):
            cand_num = lines
        elif file_paths.endswith('candidate'):
            cand = lines
        elif file_paths.endswith('src'):
            src = lines
        else:
            print(file_paths)
            raise ValueError
    
    return src, gold, cand, cand_num
    

In [ ]:
src, gold, cand, cand_num = get_result_list(join(bert_result, 'candidate_5_*'))

print("gold len : {}".format(len(gold)))
print("src len : {}".format(len(src)))
print("cand len : {}".format(len(cand)))
print("cand_num len : {}".format(len(cand_num)))

In [ ]:
assert len(gold)==len(src) and len(src)==len(cand) and len(cand_num)==len(cand)

In [ ]:
match_json_data = os.path.join(root_dir,'origin')

del_folder(origin_dir)
mkdir_p(origin_dir)

In [ ]:
origin_json_file = os.path.join(origin_dir, 'origin_data.jsonl')

with open(origin_json_file, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    for _gold, _cand, _src in zip(gold, cand, src):

        src_list = _src.split(split_symbol)
        src_list = list(map(lambda txt : txt.replace('\n',''), src_list))

        abs_list = _gold.split(split_symbol)
        abs_list = list(map(lambda txt : txt.replace('\n',''), abs_list))

        d = {"text" : src_list, "summary" : abs_list}

        writer.write(d) 

In [ ]:
index_json_file = os.path.join(origin_dir, 'index.jsonl')

with open(index_json_file, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    for _cand_num in cand_num:
        
        idx_list = _cand_num.split(split_symbol)
        idx_list = [int(idx.replace('\n','')) for idx in idx_list]
        idx_json = {"sent_id" : idx_list}
        
        writer.write(idx_json)

### BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

bart_tokenized = tok([origin_phase_txt], max_length=tok_max_length, return_tensors='pt')
bart_summarized = model.generate(bart_tokenized['input_ids'], num_beams=4, max_length=max_length, early_stopping=True)
result = [tok.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in bart_summarized]
    

### GSum

### RefSum